In [ ]:
  !pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=894a4b2cc3d8e1ed3d5f5b405698064de66e5565480761ab85008b1a7591ce34
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


# Log_Reg_dataset.csv

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('log_reg').getOrCreate()

In [ ]:
df=spark.read.csv('Log_Reg_dataset.csv',inferSchema=True,header=True)
print((df.count(), len(df.columns)))


(20000, 6)


In [ ]:
df.printSchema()


root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)



In [ ]:
df.show(10,False)

+---------+---+--------------+--------+----------------+------+
|Country  |Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|
+---------+---+--------------+--------+----------------+------+
|India    |41 |1             |Yahoo   |21              |1     |
|Brazil   |28 |1             |Yahoo   |5               |0     |
|Brazil   |40 |0             |Google  |3               |0     |
|Indonesia|31 |1             |Bing    |15              |1     |
|Malaysia |32 |0             |Google  |15              |1     |
|Brazil   |32 |0             |Google  |3               |0     |
|Brazil   |32 |0             |Google  |6               |0     |
|Indonesia|27 |0             |Google  |9               |0     |
|Indonesia|32 |0             |Yahoo   |2               |0     |
|Indonesia|31 |1             |Bing    |16              |1     |
+---------+---+--------------+--------+----------------+------+
only showing top 10 rows



In [ ]:
df.describe().show()

+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|summary| Country|              Age|   Repeat_Visitor|Platform| Web_pages_viewed|            Status|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|  count|   20000|            20000|            20000|   20000|            20000|             20000|
|   mean|    null|         28.53955|           0.5029|    null|           9.5533|               0.5|
| stddev|    null|7.888912950773227|0.500004090187782|    null|6.073903499824976|0.5000125004687693|
|    min|  Brazil|               17|                0|    Bing|                1|                 0|
|    max|Malaysia|              111|                1|   Yahoo|               29|                 1|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+



In [ ]:
df.groupBy('Country').count().show()

+---------+-----+
|  Country|count|
+---------+-----+
| Malaysia| 1218|
|    India| 4018|
|Indonesia|12178|
|   Brazil| 2586|
+---------+-----+



In [ ]:
df.groupBy('Platform').count().show()

+--------+-----+
|Platform|count|
+--------+-----+
|   Yahoo| 9859|
|    Bing| 4360|
|  Google| 5781|
+--------+-----+



In [ ]:
df.groupBy('Country').mean().show()
df.groupBy('Platform').mean().show()
df.groupBy("Status").mean().show()

+---------+------------------+-------------------+---------------------+--------------------+
|  Country|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|         avg(Status)|
+---------+------------------+-------------------+---------------------+--------------------+
| Malaysia|27.792282430213465| 0.5730706075533661|   11.192118226600986|  0.6568144499178982|
|    India|27.976854156296664| 0.5433051269288203|   10.727227476356397|  0.6212045793927327|
|Indonesia| 28.43159796354081| 0.5207751683363442|    9.985711939563148|  0.5422893742814913|
|   Brazil|30.274168600154677|  0.322892498066512|    4.921113689095128|0.038669760247486466|
+---------+------------------+-------------------+---------------------+--------------------+

+--------+------------------+-------------------+---------------------+------------------+
|Platform|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|       avg(Status)|
+--------+------------------+-------------------+----------------

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

One hot encoding


In [ ]:
Platform_indexer =StringIndexer(inputCol="Platform",
outputCol="Platform_Num").fit(df)
df = Platform_indexer.transform(df)
df.show(5,False)

+---------+---+--------------+--------+----------------+------+------------+
|Country  |Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Platform_Num|
+---------+---+--------------+--------+----------------+------+------------+
|India    |41 |1             |Yahoo   |21              |1     |0.0         |
|Brazil   |28 |1             |Yahoo   |5               |0     |0.0         |
|Brazil   |40 |0             |Google  |3               |0     |1.0         |
|Indonesia|31 |1             |Bing    |15              |1     |2.0         |
|Malaysia |32 |0             |Google  |15              |1     |1.0         |
+---------+---+--------------+--------+----------------+------+------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import OneHotEncoder
search_engine_encoder=OneHotEncoder(inputCol="Platform_Num",
outputCol='Platform_Vector').fit(df)
df = search_engine_encoder.transform(df)
df.show(3,False)

+-------+---+--------------+--------+----------------+------+------------+---------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Platform_Num|Platform_Vector|
+-------+---+--------------+--------+----------------+------+------------+---------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0         |(2,[0],[1.0])  |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0         |(2,[0],[1.0])  |
|Brazil |40 |0             |Google  |3               |0     |1.0         |(2,[1],[1.0])  |
+-------+---+--------------+--------+----------------+------+------------+---------------+
only showing top 3 rows



In [ ]:
df.groupBy('Platform_Vector').count().orderBy('count',
ascending=False).show(5,False)

+---------------+-----+
|Platform_Vector|count|
+---------------+-----+
|(2,[0],[1.0])  |9859 |
|(2,[1],[1.0])  |5781 |
|(2,[],[])      |4360 |
+---------------+-----+



One hot encoding


In [ ]:
country_indexer = StringIndexer(inputCol="Country",
outputCol="Country_Num").fit(df)
df = country_indexer.transform(df)
df.groupBy('Country').count().orderBy('count',ascending=False).show(5,False)


df.groupBy('Country_Num').count().orderBy('count',
ascending=False).show(5,False)

country_encoder = OneHotEncoder(inputCol="Country_Num",
outputCol="Country_Vector").fit(df)

df = country_encoder.transform(df)

df.select(['Country','Country_Num','Country_Vector']).show(3,False)

+---------+-----+
|Country  |count|
+---------+-----+
|Indonesia|12178|
|India    |4018 |
|Brazil   |2586 |
|Malaysia |1218 |
+---------+-----+

+-----------+-----+
|Country_Num|count|
+-----------+-----+
|0.0        |12178|
|1.0        |4018 |
|2.0        |2586 |
|3.0        |1218 |
+-----------+-----+

+-------+-----------+--------------+
|Country|Country_Num|Country_Vector|
+-------+-----------+--------------+
|India  |1.0        |(3,[1],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
+-------+-----------+--------------+
only showing top 3 rows



Vector Assembler

In [ ]:
df_assembler = VectorAssembler(inputCols=['Platform_Vector','Country_Vector','Age', 'Repeat_Visitor',
'Web_pages_viewed'], outputCol="features")
df = df_assembler.transform(df)
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- Platform_Num: double (nullable = false)
 |-- Platform_Vector: vector (nullable = true)
 |-- Country_Num: double (nullable = false)
 |-- Country_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



Splitting the dataset


In [ ]:
model_df=df.select(['features','Status'])
training_df,test_df=model_df.randomSplit([0.75,0.25])
print(training_df.count())

training_df.groupBy('Status').count().show()

14963
+------+-----+
|Status|count|
+------+-----+
|     1| 7522|
|     0| 7441|
+------+-----+



Build Logistic Model

In [ ]:
from pyspark.ml.classification import LogisticRegression
log_reg=LogisticRegression(labelCol='Status').fit(training_df)

Training results

In [ ]:
train_results=log_reg.evaluate(training_df).predictions
train_results.filter(train_results['Status']==1).filter(train_results['prediction']==1).select(['Status',
'prediction','probability']).show(10,False)

+------+----------+----------------------------------------+
|Status|prediction|probability                             |
+------+----------+----------------------------------------+
|1     |1.0       |[0.3082522414104004,0.6917477585895996] |
|1     |1.0       |[0.3082522414104004,0.6917477585895996] |
|1     |1.0       |[0.17521526788871197,0.824784732111288] |
|1     |1.0       |[0.17521526788871197,0.824784732111288] |
|1     |1.0       |[0.09196196176231414,0.9080380382376858]|
|1     |1.0       |[0.09196196176231414,0.9080380382376858]|
|1     |1.0       |[0.09196196176231414,0.9080380382376858]|
|1     |1.0       |[0.04605738608711114,0.9539426139128888]|
|1     |1.0       |[0.04605738608711114,0.9539426139128888]|
|1     |1.0       |[0.04605738608711114,0.9539426139128888]|
+------+----------+----------------------------------------+
only showing top 10 rows



Evaluation of the model

In [ ]:
results=log_reg.evaluate(test_df).predictions
results.printSchema()
results.select(['Status','prediction']).show(10,False)

root
 |-- features: vector (nullable = true)
 |-- Status: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+------+----------+
|Status|prediction|
+------+----------+
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
+------+----------+
only showing top 10 rows



Confusion Matrix

In [ ]:
tp = results[(results.Status == 1) & (results.prediction == 1)].count()
tn = results[(results.Status == 0) & (results.prediction == 0)].count()
fp = results[(results.Status == 0) & (results.prediction == 1)].count()
fn = results[(results.Status == 1) & (results.prediction == 0)].count()

Accuracy

In [ ]:
accuracy=float((tp + tn) / (results.count()))
print(accuracy)

0.9448084177089537


Recall

In [ ]:
recall = float(tp)/(tp + fn)
print(recall)

0.9426957223567393


Precision

In [ ]:
precision = float(tp) / (tp + fp)
print(precision)

0.9449838187702265


# AFFAIRS.CSV

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('log_reg').getOrCreate()

In [ ]:
df=spark.read.csv('affairs.csv',inferSchema=True,header=True)
print((df.count(), len(df.columns)))

(6366, 6)


In [ ]:
df.printSchema()
df.show(20,False)
#df.describe().show()

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [ ]:
df_assembler = VectorAssembler(inputCols=['rate_marriage','age','yrs_married', 'children',
'religious'], outputCol="features")
df = df_assembler.transform(df)
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



Splitting the Dataset

In [ ]:
model_df=df.select(['features','affairs'])
training_df,test_df=model_df.randomSplit([0.75,0.25])
print(training_df.count())

training_df.groupBy('affairs').count().show()

4757
+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1519|
|      0| 3238|
+-------+-----+



Build Logistic Model

In [ ]:
from pyspark.ml.classification import LogisticRegression
log_reg=LogisticRegression(labelCol='affairs').fit(training_df)

Training the Model

In [ ]:
train_results=log_reg.evaluate(training_df).predictions
train_results.filter(train_results['affairs']==1).filter(train_results['prediction']==1).select(['affairs',
'prediction','probability']).show(10,False)

+-------+----------+----------------------------------------+
|affairs|prediction|probability                             |
+-------+----------+----------------------------------------+
|1      |1.0       |[0.16532980306639547,0.8346701969336046]|
|1      |1.0       |[0.16532980306639547,0.8346701969336046]|
|1      |1.0       |[0.2189614443218095,0.7810385556781905] |
|1      |1.0       |[0.16131051422725298,0.838689485772747] |
|1      |1.0       |[0.27812560964472416,0.7218743903552758]|
|1      |1.0       |[0.27985770293389484,0.7201422970661051]|
|1      |1.0       |[0.27985770293389484,0.7201422970661051]|
|1      |1.0       |[0.15299691380728012,0.8470030861927199]|
|1      |1.0       |[0.265698885834292,0.734301114165708]   |
|1      |1.0       |[0.19887571142191296,0.801124288578087] |
+-------+----------+----------------------------------------+
only showing top 10 rows



Evaluation of the model

In [ ]:
results=log_reg.evaluate(test_df).predictions
results.printSchema()
results.select(['affairs','prediction']).show(60,False)

Confusion Matrix

In [ ]:
tp = results[(results.affairs == 1) & (results.prediction == 1)].count()
tn = results[(results.affairs == 0) & (results.prediction == 0)].count()
fp = results[(results.affairs == 0) & (results.prediction == 1)].count()
fn = results[(results.affairs == 1) & (results.prediction == 0)].count()

Accuracy

In [ ]:
accuracy=float((tp + tn) / (results.count()))
print(accuracy)

0.709136109384711


Recall

In [ ]:
recall = float(tp)/(tp + fn)
print(recall)

0.3258426966292135


Precision

In [ ]:
precision = float(tp) / (tp + fp)
print(precision)

0.6170212765957447
